In [1]:

import numpy as np
import tensorflow as tf

%pylab inline

import mccd
from astropy.io import fits


Populating the interactive namespace from numpy and matplotlib


runstats and/or skimage could not be imported because not installed


In [4]:
print(tf.test.gpu_device_name()) 

import matplotlib.pyplot as plt
from mccd.denoising.unets.unet import Unet
from mccd.denoising.preprocessing import eigenPSF_data_gen
from mccd.denoising.evaluate import keras_psnr, center_keras_psnr


/device:GPU:0


In [5]:

# Paths
eigenpsf_dataset_path = '/n05data/ayed/outputs/eigenpsfs/global_eigenpsfs.fits'
base_save_path = '/n05data/tliaudat/new_deepmccd/reproduce_aziz_results/testing_networks/unets/'
checkpoint_path = base_save_path + 'cp_256.h5'



In [6]:
img = fits.open(eigenpsf_dataset_path)
img = img[1].data['VIGNETS_NOISELESS']



In [9]:
size_train = np.floor(len(img)*0.90)
training, test = img[:int(size_train),:,:], img[int(size_train):,:,:]

batch_size = 32



In [10]:
training_ds = eigenPSF_data_gen(data=training,
                    snr_range=[0.1,100],
                    img_shape=(51, 51),
                    batch_size=batch_size,
                    n_shuffle=50,
                    noise_estimator=False)

test_ds = eigenPSF_data_gen(data=test,
                 snr_range=[0.1,100],
                 img_shape=(51, 51),
                 batch_size=1,
                 noise_estimator=False)



In [ ]:

n_epochs = 1000
steps = int(size_train/batch_size)

model = Unet(n_output_channels=1, kernel_size=3, layers_n_channels=[32, 64, 128, 256, 512])

adam = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(
    optimizer=adam,
    loss='mse',
    metrics=['mse', keras_psnr, center_keras_psnr],
)
